# Quantium Data Analysis

In [1]:
#Load libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

%matplotlib inline

In [2]:
'''The merged data from task 1 can be used in task 2. However, when I check the data Quantium provided in task 2, the DATE
attritbute is incorrect for task 1. So I used the new data they provided in task 2'''
data = pd.read_csv('QVI_data.csv')
data.head()

,LYLTY_CARD_NBR,DATE,STORE_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PACK_SIZE,BRAND,LIFESTAGE,PREMIUM_CUSTOMER
0,1000,2018-10-17,1,1,5,Natural Chip Compny SeaSalt175g,2,6.0,175,NATURAL,YOUNG SINGLES/COUPLES,Premium
1,1002,2018-09-16,1,2,58,Red Rock Deli Chikn&Garlic Aioli 150g,1,2.7,150,RRD,YOUNG SINGLES/COUPLES,Mainstream
2,1003,2019-03-07,1,3,52,Grain Waves Sour Cream&Chives 210G,1,3.6,210,GRNWVES,YOUNG FAMILIES,Budget
3,1003,2019-03-08,1,4,106,Natural ChipCo Hony Soy Chckn175g,1,3.0,175,NATURAL,YOUNG FAMILIES,Budget
4,1004,2018-11-02,1,5,96,WW Original Stacked Chips 160g,1,1.9,160,WOOLWORTHS,OLDER SINGLES/COUPLES,Mainstream


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264834 entries, 0 to 264833
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   LYLTY_CARD_NBR    264834 non-null  int64  
 1   DATE              264834 non-null  object 
 2   STORE_NBR         264834 non-null  int64  
 3   TXN_ID            264834 non-null  int64  
 4   PROD_NBR          264834 non-null  int64  
 5   PROD_NAME         264834 non-null  object 
 6   PROD_QTY          264834 non-null  int64  
 7   TOT_SALES         264834 non-null  float64
 8   PACK_SIZE         264834 non-null  int64  
 9   BRAND             264834 non-null  object 
 10  LIFESTAGE         264834 non-null  object 
 11  PREMIUM_CUSTOMER  264834 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 24.2+ MB


In [4]:
#Change data type to save memory
int64 =  data.select_dtypes(include=['int64']).columns.tolist() 
data[int64] = data[int64].astype('int32')

data['TOT_SALES'] = data['TOT_SALES'].astype('float16') #Change float64 to float16

#Change DATE to datetime data
data['DATE'] = pd.to_datetime(data['DATE'],format = '%Y-%m-%d')

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264834 entries, 0 to 264833
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   LYLTY_CARD_NBR    264834 non-null  int32         
 1   DATE              264834 non-null  datetime64[ns]
 2   STORE_NBR         264834 non-null  int32         
 3   TXN_ID            264834 non-null  int32         
 4   PROD_NBR          264834 non-null  int32         
 5   PROD_NAME         264834 non-null  object        
 6   PROD_QTY          264834 non-null  int32         
 7   TOT_SALES         264834 non-null  float16       
 8   PACK_SIZE         264834 non-null  int32         
 9   BRAND             264834 non-null  object        
 10  LIFESTAGE         264834 non-null  object        
 11  PREMIUM_CUSTOMER  264834 non-null  object        
dtypes: datetime64[ns](1), float16(1), int32(6), object(4)
memory usage: 16.7+ MB


The client has selected store numbers 77, 86 and 88 as trial stores and want 
control stores to be established stores that are operational for the entire 
observation period.
We would want to match trial stores to control stores that are similar to the trial
store prior to the trial period of Feb 2019 in terms of :
- Monthly overall sales revenue
- Monthly number of customers
- Monthly number of transactions per customer

Let's first create the metrics of interest and filter to stores that are present 
throughout the pre-trial period.

In [6]:
#Add a new month ID column in the data with format yyyymm
data['MONTH_ID'] = data['DATE'].dt.to_period('M')
data.head()

,LYLTY_CARD_NBR,DATE,STORE_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PACK_SIZE,BRAND,LIFESTAGE,PREMIUM_CUSTOMER,MONTH_ID
0,1000,2018-10-17,1,1,5,Natural Chip Compny SeaSalt175g,2,6.000000,175,NATURAL,YOUNG SINGLES/COUPLES,Premium,2018-10
1,1002,2018-09-16,1,2,58,Red Rock Deli Chikn&Garlic Aioli 150g,1,2.699219,150,RRD,YOUNG SINGLES/COUPLES,Mainstream,2018-09
2,1003,2019-03-07,1,3,52,Grain Waves Sour Cream&Chives 210G,1,3.599609,210,GRNWVES,YOUNG FAMILIES,Budget,2019-03
3,1003,2019-03-08,1,4,106,Natural ChipCo Hony Soy Chckn175g,1,3.000000,175,NATURAL,YOUNG FAMILIES,Budget,2019-03
4,1004,2018-11-02,1,5,96,WW Original Stacked Chips 160g,1,1.900391,160,WOOLWORTHS,OLDER SINGLES/COUPLES,Mainstream,2018-11


As we only considered stores that have sales every month over July 2018 to June 2019, stores that do not satisfy the condition need to be dropped

In [7]:
count_months = data.groupby('STORE_NBR')['MONTH_ID'].nunique()
count_months = count_months[count_months < 12]
store_to_drop = pd.DataFrame(count_months).index

For each store and month calculate total sales, number of customers, transactions per customer, chips per customer and the average price per unit.

In [13]:
data = data.loc[~data.STORE_NBR.isin(store_to_drop)]
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 264645 entries, 0 to 264833
Data columns (total 20 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   LYLTY_CARD_NBR            264645 non-null  int32         
 1   DATE                      264645 non-null  datetime64[ns]
 2   STORE_NBR                 264645 non-null  int32         
 3   TXN_ID                    264645 non-null  int32         
 4   PROD_NBR                  264645 non-null  int32         
 5   PROD_NAME                 264645 non-null  object        
 6   PROD_QTY                  264645 non-null  int32         
 7   TOT_SALES                 264645 non-null  float16       
 8   PACK_SIZE                 264645 non-null  int32         
 9   BRAND                     264645 non-null  object        
 10  LIFESTAGE                 264645 non-null  object        
 11  PREMIUM_CUSTOMER          264645 non-null  object        
 12  MO

In [9]:
#total sales for each store and month
data['MONTHLY_SALES'] = data.groupby(['STORE_NBR','MONTH_ID'])['TOT_SALES'].transform('sum')

#number of customer for each store and month
data['MONTHLY_CUSTOMER'] = data.groupby(['STORE_NBR','MONTH_ID'])['LYLTY_CARD_NBR'].transform('nunique')

#transactions per customer for each store and month
data['MONTHLY_TRANSACTION'] = data.groupby(['STORE_NBR','MONTH_ID'])['TXN_ID'].transform('count') #Count number of transaction
data['TRANSACTION_PER_CUSTOMER'] = data['MONTHLY_TRANSACTION']/data['MONTHLY_CUSTOMER'] #Number of transaction per customer

#chips per customer for each store and month
data['MONTHLY_QTY'] = data.groupby(['STORE_NBR','MONTH_ID'])['PROD_QTY'].transform('sum') #Total product quantity
data['QTY_PER_CUSTOMER'] = data['MONTHLY_QTY']/data['MONTHLY_CUSTOMER'] #No of chips per customer

#average price per unit
data['AVERAGE_PRICE'] =data['MONTHLY_SALES']/data['MONTHLY_QTY']

In [10]:
data.head()

,LYLTY_CARD_NBR,DATE,STORE_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PACK_SIZE,BRAND,LIFESTAGE,PREMIUM_CUSTOMER,MONTH_ID,MONTHLY_SALES,MONTHLY_CUSTOMER,MONTHLY_TRANSACTION,TRANSACTION_PER_CUSTOMER,MONTHLY_QTY,QTY_PER_CUSTOMER,AVERAGE_PRICE
0,1000,2018-10-17,1,1,5,Natural Chip Compny SeaSalt175g,2,6.000000,175,NATURAL,YOUNG SINGLES/COUPLES,Premium,2018-10,188.125,44,45,1.022727,58,1.318182,3.243534
1,1002,2018-09-16,1,2,58,Red Rock Deli Chikn&Garlic Aioli 150g,1,2.699219,150,RRD,YOUNG SINGLES/COUPLES,Mainstream,2018-09,278.750,59,62,1.050847,75,1.271186,3.716667
2,1003,2019-03-07,1,3,52,Grain Waves Sour Cream&Chives 210G,1,3.599609,210,GRNWVES,YOUNG FAMILIES,Budget,2019-03,192.875,45,49,1.088889,58,1.288889,3.325431
3,1003,2019-03-08,1,4,106,Natural ChipCo Hony Soy Chckn175g,1,3.000000,175,NATURAL,YOUNG FAMILIES,Budget,2019-03,192.875,45,49,1.088889,58,1.288889,3.325431
4,1004,2018-11-02,1,5,96,WW Original Stacked Chips 160g,1,1.900391,160,WOOLWORTHS,OLDER SINGLES/COUPLES,Mainstream,2018-11,192.625,46,47,1.021739,57,1.239130,3.379386


Filter to the pretrial period

In [11]:
pre_trial = data[data.MONTH_ID < '2019-02']

Now we need to work out a way of ranking how similar each potential control store is to the trial store. We can calculate how correlated the performance of each store is to the trial store.

In [14]:
metric_columns = ['STORE_NBR','MONTH_ID','MONTHLY_SALES','MONTHLY_CUSTOMER','TRANSACTION_PER_CUSTOMER','QTY_PER_CUSTOMER',
                 'AVERAGE_PRICE']
input_data = pre_trial.loc[:,metric_columns].set_index(['STORE_NBR','MONTH_ID'])
input_data.sort_values(by = ['STORE_NBR','MONTH_ID'],ascending = True, inplace = True)
input_data.drop_duplicates(inplace = True, keep = 'first')

#Function to calculate correlation for a measure, looping through each control store
def cal_corr(input_data, metric_col, store_comparison):
    #Select Store_NBR, Month_ID, and the metric used to calculate correlation on
    metric = input_data[[metric_col]]

    #Loop through each store, calculate correlation
    trial_store = metric.loc[store_comparison]
    series = []
    for i in metric.index:
        corr = trial_store.corrwith(metric.loc[i[0]])
        series.append(corr)
    
    result = pd.DataFrame(series)
    result.index = metric.index
    result.index = [i[0] for i in result.index] #Select only STORE_NBR as index
    result.index.name = 'STORE_NBR'
    result = result.abs() #Make the correlation ranges from 0 to 1
    result.drop_duplicates(inplace = True)
    result.sort_values(by = metric_col, ascending = False, inplace = True)
    return result

In [17]:
#Test if the function work
a = cal_corr(input_data,'MONTHLY_SALES',77)
a.head()

,MONTHLY_SALES
STORE_NBR,
77,0.998847
71,0.913691
233,0.903432
119,0.867842
258,0.854345


It works! Now let's create an other function to calculate magnitude

In [20]:
def cal_magnitude(input_data,metric_1, metric_2,store_comparison):
    a = cal_corr(input_data,metric_1,store_comparison)
    b = cal_corr(input_data,metric_2,store_comparison)
    result = pd.merge(a,b,left_index = True, right_index = True)
    result['MAGNITUDE'] = result.mean(axis = 1)
    return result

Now let's use the functions to find the control stores! We'll select control stores based on how similar monthly total sales in dollar amounts and monthly number of customers are to the trial stores.

## Trial Store 77